# Benchmarking Leiden Community Detection Algorithms

This notebook downloads the `cit-Patents` dataset from SNAP and benchmarks the performance of the Leiden community detection algorithm across several popular Python graph libraries.

## 1. Initial Setup and Data Loading

First, we'll import the basic libraries needed for data handling and define a function to download and load the dataset into a pandas DataFrame and a base NetworkX graph.

In [ ]:
from pathlib import Path
import requests
import gzip
import pandas as pd
import networkx as nx

In [ ]:
def get_edgelist():
    """Downloads and extracts the cit-Patents dataset."""
    # From https://snap.stanford.edu/data/cit-Patents.html
    url = "https://snap.stanford.edu/data/cit-Patents.txt.gz"
    gz_file_name = Path(url.split("/")[-1])
    csv_file_name = Path(gz_file_name.stem)

    if csv_file_name.exists():
        print(f"{csv_file_name} already exists, not downloading.")
    else:
        print(f"Downloading {url}...", end="", flush=True)
        req = requests.get(url)
        open(gz_file_name, "wb").write(req.content)
        print("done")

        print(f"Unzipping {gz_file_name}...", end="", flush=True)
        with gzip.open(gz_file_name, "rb") as gz_in:
            with open(csv_file_name, "wb") as txt_out:
                txt_out.write(gz_in.read())
        print("done")

    print("Reading CSV to DataFrame...", end="", flush=True)
    pandas_edgelist = pd.read_csv(
        csv_file_name.name,
        skiprows=4,
        delimiter="\t",
        names=["src", "dst"],
        dtype={"src":"int32", "dst":"int32"},
    )
    print("done")
    return pandas_edgelist

In [ ]:
pandas_edgelist = get_edgelist()
G = nx.from_pandas_edgelist(pandas_edgelist, source="src", target="dst")
print(f"\nGraph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

---
## 2. NetworkX + nx-cugraph

In [ ]:
%%time
c = nx.community.leiden_communities(G)
print(f"Number of communities: {len(c)}")

---
## 3. Graspologic

In [ ]:
from graspologic.partition import leiden, hierarchical_leiden

In [ ]:
%%time
print("Timing leiden using Graspologic...")
c = leiden(G)
print(f"Number of communities: {len(set(c.values()))}")

In [ ]:
%%time
print("Timing hierarchical_leiden using Graspologic...")
c = hierarchical_leiden(G)
print(f"Number of communities: {len(set([cl.cluster for cl in c]))}")

---
## 4. cuGraph

In [ ]:
import cugraph

In [ ]:
%%time
print("Timing cuGraph graph creation from pandas edgelist...")
Gcg = cugraph.from_pandas_edgelist(pandas_edgelist, source="src", destination="dst")

In [ ]:
%%time
print("Timing leiden using cuGraph...")
(df, modularity) = cugraph.leiden(Gcg)
del Gcg
print(f"Number of communities: {len(df['partition'].unique())}")

---
## 5. igraph

In [ ]:
import igraph

In [ ]:
%%time
print("Timing igraph graph creation from networkx graph...")
Gig = igraph.Graph.from_networkx(G)

In [ ]:
%%time
print("Timing leiden using igraph...")
c = Gig.community_leiden(objective_function="modularity", n_iterations=-1)
print(f"Number of communities: {len(c)}")

---
## 6. leidenalg

In [ ]:
import leidenalg

In [ ]:
%%time
print("Timing leiden using leidenalg...")
c = leidenalg.find_partition(Gig, leidenalg.ModularityVertexPartition)
print(f"Number of communities: {len(c)}")

---
## 7. pylibcugraph

In [ ]:
import pylibcugraph as plc
import cupy as cp

In [ ]:
%%time
print("Timing pylibcugraph graph creation from pandas edgelist...")
srcs = cp.asarray(pandas_edgelist["src"], dtype="int32")
dsts = cp.asarray(pandas_edgelist["dst"], dtype="int32")
unique_vertices = cp.unique(cp.concatenate((srcs, dsts)))
resource_handle = plc.ResourceHandle()
Gplc = plc.SGGraph(
    resource_handle,
    graph_properties=plc.GraphProperties(is_symmetric=True, is_multigraph=False),
    src_or_offset_array=srcs,
    dst_or_index_array=dsts,
    weight_array=cp.ones(len(srcs), dtype="float32"),
    store_transposed=False,
    renumber=True,
    do_expensive_check=False,
    vertices_array=unique_vertices,
    drop_multi_edges=True,
    symmetrize=True,
    input_array_format="COO",
)
del srcs, dsts, unique_vertices

In [ ]:
%%time
print("Timing leiden using pylibcugraph...")
(vertices, clusters, modularity) = plc.leiden(
    resource_handle,
    graph=Gplc,
    max_level=100,
    resolution=1.0,
    theta=1.0,
    do_expensive_check=False
)
del resource_handle, Gplc
print(f"Number of communities: {cp.unique(clusters).shape[0]}")

---
## 8. scikit-network

Note: The original script mentioned that `sknetwork.clustering.leiden` did not finish after ~12 hours. We are using `Louvain` instead for this benchmark, as suggested.

In [ ]:
import sknetwork

In [ ]:
# Setup for scikit-network
edgelist = list(pandas_edgelist.itertuples(index=False, name=None))
adjacency = sknetwork.utils.from_edge_list(edgelist, directed=False, reindex=True)["adjacency"]

In [ ]:
%%time
print("Timing louvain using sknetwork...")
louvain = sknetwork.clustering.Louvain()
c = louvain.fit_predict(adjacency)
print(f"Number of communities: {len(set(c))}")

---
## 9. cdlib

In [ ]:
import cdlib.algorithms
import cdlib.utils
import igraph

In [ ]:
%%time
print("Timing leiden using cdlib (on NetworkX graph)...")
c = cdlib.algorithms.leiden(G)
print(f"Number of communities: {len(c.communities)}")

In [ ]:
# cdlib can be slow when converting large graphs, so we separate this step
Gig_cdlib = cdlib.utils.convert_graph_formats(G, desired_format=igraph.Graph, directed=False)

In [ ]:
%%time
print("Timing leiden using cdlib (on igraph graph)...")
c = cdlib.algorithms.leiden(Gig_cdlib)
print(f"Number of communities: {len(c.communities)}")